In [2]:
# imports
import pandas as pd

In [35]:
# read the data
class_data, stats_data = pd.read_csv('address_clust.csv'), pd.read_csv('address_stats.csv')

In [36]:
# merge dataframe, name 'zero' claster and drop excess columns
data_all = class_data.merge(stats_data, how='outer').drop(columns=['id','address_id']).fillna({'cluster_id':0})
data_all.head()

,cluster_id,transaction_id,received,sent
0,1.0,29319673,1003590.0,0.0
1,1.0,29338157,0.0,1003590.0
2,1.0,29319205,1259370.0,0.0
3,1.0,29338157,0.0,1259370.0
4,1.0,29318455,21009000.0,0.0


In [37]:
# group data by transaction and count sent & recived money
b = data_all.groupby(['transaction_id', 'cluster_id']).agg({'received':'sum', 'sent' : 'sum'}).reset_index()
b.head()

,transaction_id,cluster_id,received,sent
0,19162323,0.0,1821052.0,2871052.0
1,19162323,1.0,1000000.0,0.0
2,19166856,0.0,990000.0,0.0
3,19166856,1.0,0.0,1000000.0
4,19174421,0.0,4590000.0,4800000.0


In [38]:
c = b.pivot(index='transaction_id', columns='cluster_id').fillna(0)
c.head()

received                       sent                
cluster_id            0.0        1.0  2.0        0.0        1.0  2.0
transaction_id                                                      
19162323        1821052.0  1000000.0  0.0  2871052.0        0.0  0.0
19166856         990000.0        0.0  0.0        0.0  1000000.0  0.0
19174421        4590000.0   200000.0  0.0  4800000.0        0.0  0.0
19190163         150000.0        0.0  0.0        0.0   200000.0  0.0
19205422        1671052.0  1000000.0  0.0  2721052.0        0.0  0.0

In [39]:
# create dataframe with 1 row : all transaction information (_r for recived, _s for sent)
c.columns = c.columns.droplevel(0)
c.columns = ['zero_r', 'one_r', 'two_r', 'zero_s', 'one_s', 'two_s']
c.reset_index
c.head()

,zero_r,one_r,two_r,zero_s,one_s,two_s
transaction_id,,,,,,
19162323,1821052.0,1000000.0,0.0,2871052.0,0.0,0.0
19166856,990000.0,0.0,0.0,0.0,1000000.0,0.0
19174421,4590000.0,200000.0,0.0,4800000.0,0.0,0.0
19190163,150000.0,0.0,0.0,0.0,200000.0,0.0
19205422,1671052.0,1000000.0,0.0,2721052.0,0.0,0.0


In [40]:
c.loc[(c.two_r > 0)&(c.one_s >0),['two_r']]

,two_r
transaction_id,
21645495,24022000.0
21646439,8014000.0
21648692,192094000.0
21650595,128056000.0
21682999,4268000.0
...,...
23619127,8004000.0
23619420,4924000.0
23620126,2452000.0


In [41]:
#__count money gained from one claster to other

def money_clasters(data, new_column_name, claster_from, claster_to):
    data.loc[:, new_column_name] = 0
    data.loc[(data[claster_from] > 0) & (data[claster_to] > 0),
             [new_column_name]] = data.loc[:, claster_to]
    

In [42]:
money_clasters(c, 'one_to_two', 'one_s', 'two_r')
money_clasters(c, 'two_to_one', 'two_s', 'one_r')
money_clasters(c, 'zero_to_two', 'zero_s', 'two_r')
money_clasters(c, 'two_to_zero', 'two_s', 'zero_r')

In [43]:
# count fee

# 1-claster fee
c.loc[:, 'one_fee'] = 0
c.loc[(c.one_s > 0),
['one_fee']] = c.loc[:, 'one_s'] - (c.loc[:, 'one_r'] + c.loc[:, 'two_r'] + c.loc[:, 'zero_r'])

# 2-claster fee
c.loc[:, 'two_fee'] = 0
c.loc[(c.two_s > 0),
['two_fee']] = c.loc[:, 'two_s'] - (c.loc[:, 'one_r'] + c.loc[:, 'two_r'] + c.loc[:, 'zero_r'])

In [44]:
c.loc[29319673].to_frame()

,29319673
zero_r,0.0
one_r,6003590.0
two_r,0.0
zero_s,0.0
one_s,6013590.0
two_s,0.0
one_to_two,0.0
two_to_one,0.0
zero_to_two,0.0
two_to_zero,0.0


In [45]:
# count sum and change satoshi to bitcoin

bitcoin_spent = c.loc[:, ['one_to_two', 'two_to_one', 'zero_to_two',
                    'two_to_zero','one_fee', 'two_fee']].sum().to_frame().rename(columns={0:'bitcoin'})
bitcoin_spent.loc[:, 'bitcoin'] = bitcoin_spent.loc[:, 'bitcoin'] / 10**8
bitcoin_spent

,bitcoin
one_to_two,129.243240
two_to_one,135.820000
zero_to_two,9.740139
two_to_zero,4.869840
one_fee,0.667144
two_fee,0.745667
